<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/NLP/Semantic_Search_with_Sentence_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Build a semantic search engine that ranks documents/passages by semantic
similarity to a user query using pretrained sentence embeddings.

In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Prepare the Document Corpus

In [4]:
documents = [
  "How do I reset my password?",
  "What is the refund policy for returned items?",
  "How can I track my order?",
  "Where are you located?",
  "How do I contact customer service?",
  "Can I cancel my subscription anytime?"
]

 Load Sentence-BERT Model

In [2]:
from sentence_transformers import SentenceTransformer
# Load a lightweight model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Encode the Documents

In [5]:
# Convert all documents into sentence embeddings
doc_embeddings = model.encode(documents, convert_to_tensor=True)

Take a User Query and Encode It

In [7]:
text = input("Enter your query: ")
query_embedding = model.encode(text, convert_to_tensor=True)

Enter your query: how can i speak to support


Compute Semantic Similarity

In [8]:
from sentence_transformers.util import cos_sim
# Calculate cosine similarity between query and all documents
similarities = cos_sim(query_embedding, doc_embeddings)[0]
# Rank results by similarity
import numpy as np
top_k = 3
top_results = np.argsort(-similarities)[:top_k]
# Show top matches
print(f"\nQuery: {text}\nTop {top_k} results:")
for idx in top_results:
  print(f"{documents[idx]} (score: {similarities[idx]:.4f})")


Query: how can i speak to support
Top 3 results:
How do I contact customer service? (score: 0.4060)
How can I track my order? (score: 0.1779)
Can I cancel my subscription anytime? (score: 0.1545)


Wrap in a Search Function

In [9]:
def semantic_search(query, docs, embeddings, model, top_k=3):
  query_emb = model.encode(query, convert_to_tensor=True)
  scores = cos_sim(query_emb, embeddings)[0]
  top_k_idx = np.argsort(-scores)[:top_k]
  return [(docs[i], float(scores[i])) for i in top_k_idx]

In [11]:
# Try it
results = semantic_search("I need help with my account", documents, doc_embeddings, model)
for text, score in results:
  print(f"> {text} (score: {score:.4f})")

> How do I reset my password? (score: 0.5667)
> How do I contact customer service? (score: 0.3736)
> Can I cancel my subscription anytime? (score: 0.3236)
